In [1]:
#to get auto matic reloading and inline editing 
%reload_ext autoreload
%autoreload 2 
%matplotlib inline

#my fast ai libraries
from fastai.imports import  * 
from fastai.transforms import *
from fastai.torch_imports import *
from fastai.conv_learner import * 
from fastai.model import *
from fastai.dataset import * 
from fastai.sgdr import *
from fastai.plots import * 
torch.cuda.set_device(0)

In [2]:
PATH = "data/dogbreeds/"
sz = 224
bs = 58
arch = inception_4

In [3]:
label_csv = f'{PATH}labels.csv'# load csv and get validation set
n = len(list(open(label_csv))) 
val_idx = get_cv_idxs(n)  

In [ ]:
!ls {PATH}

In [ ]:
label_df = pd.read_csv(label_csv) # get header of the cvs file, see whats in it 
label_df.head()

In [ ]:
"""
createa a pivot table to see what data is in there, sorting by length 
breed is the main index 
"""
label_df.pivot_table(index='breed', aggfunc=len).sort_values('id', ascending=False) 

In [4]:
"""Here is the function to get the data from teh CSV file
We can first give it the path for the CSV, the folder to use, 
test_name (for kaggle), validation indexes, suffix if 
there is none in the data transforms and batch size
"""
def getdata(sz, bs):
    tfm = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', val_idxs = val_idx, tfms=tfm,
                                        suffix= '.jpg', test_name= 'test', 
                                        bs=bs, num_workers = 4)
    return data if sz >300 else data.resize(340, 'tmp')

In [5]:
data = getdata(sz, bs) #get the data 

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

In [ ]:
fn = PATH+data.trn_ds.fnames[0]; fn

In [ ]:
img = PIL.Image.open(fn); img

In [ ]:
img.size

In [ ]:
#need to check image size 
size_d = {k: PIL.Image.open(PATH+k).size for k in data.trn_ds.fnames}

In [ ]:
row_sz, col_sz = list(zip(*size_d.values())) # link the lists of rows and that 

In [ ]:
row_sz = np.array(row_sz); col_sz = np.array(col_sz)

In [ ]:
row_sz[:5]

In [ ]:
plt.hist(row_sz[row_sz<1000])

In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [ ]:
lrf = learn.lr_find()
learn.predict()

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.sched.plot()

In [ ]:
learn.fit(1e-1, 5)

In [ ]:
from sklearn import metrics

In [6]:
data = getdata(299,bs)
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

In [ ]:
learn.fit(1e-2, 5)

In [ ]:
learn.precompute = False

In [ ]:
learn.fit(1e-2, 3, cycle_len =1)

In [ ]:
learn.save('244_inc')

In [ ]:
learn.load('244_inc')

In [ ]:
learn.set_data(getdata(299,bs))
learn.freeze()

In [ ]:
learn.fit(1e-1,3)

In [ ]:
learn.fit(1e-2, 3, cycle_len = 1, cycle_mult =2)

In [ ]:
log_preds, y = learn.TTA()
probs = np.exp(log_preds)
accuracy(log_preds,y), metrics.log_loss(y, probs)

In [ ]:
learn.save('299_inc')

In [8]:
learn.precompute = False
learn.load('299_inc')

In [9]:
learn.fit(1e-2,5,cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=5), HTML(value='')))

  1%|          | 1/141 [00:04<09:38,  4.13s/it, loss=0.344]

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[0.      0.23399 0.2272  0.93295]                            
[1.      0.23445 0.22529 0.9363 ]                            
[2.      0.24982 0.22661 0.93487]                            
[3.      0.24024 0.22597 0.93678]                            
[4.      0.21914 0.22788 0.93487]                            



In [10]:
learn.save('299_final')


In [ ]:
log_preds, y = learn.TTA()
probs = np.exp(log_preds)
accuracy(log_preds,y), metrics.log_loss(y, probs)

In [ ]:
log_preds, y = learn.TTA(is_test = True)
probs = np.exp(log_preds)
accuracy(log_preds,y), metrics.log_loss(y, probs)

In [ ]:
learn.fit(1e-2, 2, cycle_len = 1,wds=0.025)

In [ ]:
learn.freeze()
learn.precompute = False
learn.load('299_final')

In [ ]:
prob = learn.predict(is_test=True)

In [ ]:
test_log_preds,y = learn.TTA(is_test=True)
test_probs = np.exp(test_log_preds)

In [ ]:
probs


In [ ]:
test_probs = np.exp(probs)

In [ ]:
test_probs.shape

In [ ]:
!ls data/dogbreeds/test | wc -l

In [11]:
log_preds, y = learn.TTA(is_test = True)
probs = np.mean(np.exp(log_preds), 0)

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [12]:
"""
submit = pd.DataFrame(probs)
submit.columns = data.classes
submit.insert(0,'id',[Id[5:-4] for Id in data.test_ds.fnames]) #skip prefix and suffix
"""
probs2 = np.exp(log_preds)
probs.shape


(120,)

In [13]:
submit = pd.DataFrame(probs2)
submit.columns = data.classes
submit.insert(0, 'id', [Id[5:-4] for Id in data.test_dl.dataset.fnames])
submit.shape


(10357, 121)

In [14]:
submit.to_csv(f'{PATH}submit_2.csv',index=False)

In [18]:
from sklearn import metrics

metrics.log_loss(y, probs)

ValueError: Found input variables with inconsistent numbers of samples: [120, 10357]

In [ ]:
learn.fit(1e-2, 2)

In [ ]:
log_preds, y = learn.TTA(is_test = True)
probs = np.mean(np.exp(log_preds), 0)
probs2 = np.exp(log_preds)
probs.shape

In [ ]:
submit = pd.DataFrame(probs2)
submit.columns = data.classes
submit.insert(0, 'id', [Id[5:-4] for Id in data.test_dl.dataset.fnames])
submit.shape
submit.to_csv(f'{PATH}submit.csv',index=False)